# Tarea 1: Bayes ingenuo

Este programa clasifica correos electrónicos como spam o ham utilizando el alrogítmo de bayes ingenuo.

In [1]:
usePackage <- function(p) 
{
  if (!is.element(p, installed.packages()[,1]))
    install.packages(p, repos = "https://cran.itam.mx/")
  suppressPackageStartupMessages(require(p, character.only = TRUE, quietly  = TRUE))
}

In [2]:
usePackage('R.utils')
usePackage('tm')

## Descarga los datos 

In [3]:
download.mails <- function(url, dir_name, file_name){

  if (!file.exists(dir_name)) {
    dir.create(dir_name)  #El directorio se crea donde este corriendo en Notebook
  }
  
    # destfile es como nombramos nuestro archivo
    #file.path va a poner un \ entre dir_name y el file_name
  download.file(url, destfile=file.path(dir_name, paste0(file_name,".tar.bz2")) )
    # Ahora lo vamos a descompactar
  bunzip2(file.path(dir_name, paste0(file_name,".tar.bz2")))
    # Quita el formato tar, saca todo de un archivo y lo pone en varios
    # Todo lo que explote va a quedar dentro de dir_name
  untar(file.path(dir_name, paste0(file_name,".tar")), exdir = dir_name)
    # Este paso es por limpieza, quitamos el comprimido
  if (file.exists(file.path(dir_name, paste0(file_name,".tar")))) {
    file.remove(file.path(dir_name, paste0(file_name,".tar")))
  }
  
}

In [4]:
dir_name <- "data"
file_name <- "easy_ham_2"
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_easy_ham_2.tar.bz2"

download.mails(url, dir_name, file_name)

[1] TRUE

In [5]:
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_hard_ham.tar.bz2"
file_name <- "hard_ham"

download.mails(url, dir_name, file_name)

[1] TRUE

In [6]:
url <- "http://spamassassin.apache.org/old/publiccorpus/20030228_spam_2.tar.bz2"
file_name <- "spam_2"

download.mails(url, dir_name, file_name)

[1] TRUE

## Preprocesamiento de los correos electrónicos

In [7]:
# Hacemos una función que lea el mensaje del archivo que se le pase como parámetro
# asumimos que el archivo contiene un correo

lee_mensaje <- function(correo) {
  fd <- file(correo, open = "rt") #Abre el archivo
  lineas <- readLines(fd, warn=FALSE)  #Leemos el archivo, es un vector de caracteres
  close(fd) #cierro el archivo
    #Ahora vamos a buscar la primer linea en blanco, pues despues de esta empieza el correo
    # lineas == "" va a regresar un vector de booleanos con TRUE y FALSE
    #Con which pides que te traiga los indices del vector donde hay TRUE
    # which(lineas == "")[1] + 1 a partir de esta linea ya es el cuerpo del correo
  mensaje <- lineas[seq(which(lineas == "")[1] + 1, length(lineas), 1)]
  return (paste(mensaje, collapse = "\n")) #Colapsa el vector de caracteres para tener un unico texto
}

In [43]:
# Creamos variables con los directorios donde se encuentran los datos
trayectoria_spam     <- file.path(dir_name, "spam_2")
trayectoria_easyham  <- file.path(dir_name, "easy_ham_2")
trayectoria_hardham  <- file.path(dir_name, "hard_ham")

### SPAM: dividimos la base de datos en training y testing

In [44]:
# Leemos el directorio donde se encuentran los correos clasificados como spam
archivos_correos_spam <- dir(trayectoria_spam) #dir me regresa el contenido del directorio
#Se genera un vector con los nombres de todos los archivos

# quitamos el guión llamado cmds
archivos_correos_spam <- archivos_correos_spam[which(archivos_correos_spam!="cmds")] #[1:250]

#### Hacemos la división de la base tomando primero un sample ####
sample_spam<-sample(archivos_correos_spam)

archivos_correos_spam_training<-sample_spam[1:1000]  #Los primeros 1000 mails barajeados se usaran para training
archivos_correos_spam_testing<-tail(sample_spam,length(sample_spam)-1000) #El resto de los mails se utilizaran para testing

todo_spam <- sapply(archivos_correos_spam_training, #Obtenemos todo_spam con la base de entrenamiento
                   function(p) lee_mensaje(file.path(trayectoria_spam, p)))
                    
todo_spam <- enc2utf8(todo_spam) #Cambias la coddificacion a utf8

### Easy ham: dividimos la base de datos en training y testing

In [45]:
# Leemos el directorio donde se encuentran los correos clasificados como ham fácilmente identificables
archivos_correos_easy_ham <- dir(trayectoria_easyham)

# quitamos el guión llamado cmds
archivos_correos_easy_ham <- archivos_correos_easy_ham[which(archivos_correos_easy_ham!="cmds")] #[1:250]

### Hacemos la división de la base tomando primero un sample ####
sample_easyHam<-sample(archivos_correos_easy_ham)

archivos_correos_easy_ham_training<-sample_easyHam[1:1000] #Los primeros 1000 mails barajeados se usaran para training
archivos_correos_easy_ham_testing<-tail(sample_easyHam,length(sample_easyHam)-1000) #El resto de los mails se utilizaran para testing

todo_easy_ham <- sapply(archivos_correos_easy_ham_training,
                    function(p) lee_mensaje(file.path(trayectoria_easyham, p)))

todo_easy_ham <- enc2utf8(todo_easy_ham)

## Preparación de corpus y bolsa de palabras

In [46]:
obtiene_TermDocumentMatrix <- function (vector_correos) {
  control <- list(stopwords = TRUE, # Palabras presentes en todas partes: de, y, etc
                removePunctuation = TRUE, 
                removeNumbers = TRUE,
                minDocFreq = 2) #Que deje las palabras que por lo menos aparezcan dos veces
    #VectorSource le dices que es un vector que es la fuente del corpus
    #Creamos un objeto de tipo corpus con la funcion Corpus
  corpus <- Corpus(VectorSource(vector_correos))
  return(TermDocumentMatrix(corpus, control))
}

### Spam

In [47]:
spam_TDM <- obtiene_TermDocumentMatrix(todo_spam) #Es un objeto de tipo TermDocumentMatrix

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento SPAM
matriz_spam <- as.matrix(spam_TDM) #Aqui se tiene la matriz: en renglones palabras, en columnas archivo

conteos_spam <- rowSums(matriz_spam)
df_spam <- data.frame(cbind(names(conteos_spam),
                            as.numeric(conteos_spam)),
                      stringsAsFactors = FALSE)
names(df_spam) <- c("terminos", "frecuencia")
df_spam$frecuencia <- as.numeric(df_spam$frecuencia)
ocurrencias_spam <- sapply(1:nrow(matriz_spam),
                          function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                          {
                            length(which(matriz_spam[i, ] > 0)) / ncol(matriz_spam)
                          })

# Vamos a obtener la proba de que salga cada palabra de manera frecuentista
densidad_spam <- df_spam$frecuencia/sum(df_spam$frecuencia,na.rm = TRUE)

df_spam <- transform(df_spam,  #Agrego columnas de densidad y ocurrencias al df
                     densidad = densidad_spam,
                     ocurrencias = ocurrencias_spam)

# IMPORTANTE: densidad es la proba de que x palabra aparezca en spam

### Easy ham

In [48]:
easy_ham_TDM <- obtiene_TermDocumentMatrix(todo_easy_ham)

# Crea un data frame que provee el conjunto de caracteristicas de los datos de entrenamiento easy ham
matriz_easy_ham <- as.matrix(easy_ham_TDM)

conteos_easy_ham <- rowSums(matriz_easy_ham)
df_easy_ham <- data.frame(cbind(names(conteos_easy_ham),
                            as.numeric(conteos_easy_ham)),
                      stringsAsFactors = FALSE)
names(df_easy_ham) <- c("terminos", "frecuencia")
df_easy_ham$frecuencia <- as.numeric(df_easy_ham$frecuencia)
ocurrencias_easy_ham <- sapply(1:nrow(matriz_easy_ham),
                           function(i) # Obtiene la proporcion de documentos que contiene cada palabra
                           {
                             length(which(matriz_easy_ham[i, ] > 0)) / ncol(matriz_easy_ham)
                           })
densidad_easy_ham <- df_easy_ham$frecuencia/sum(df_easy_ham$frecuencia,na.rm = TRUE)

df_easy_ham <- transform(df_easy_ham,
                     densidad = densidad_easy_ham,
                     ocurrencias = ocurrencias_easy_ham)

## Cálculo de probabilidad a posteriori

In [49]:

# Bayes ingenuo te dice que multipliques las probas, pero para cada palabra, estas son muy pequeñas
# Tendriamos una proba muy cercana a cero, pues la precision de la compu no es tan grande
# Si sumamos las probas no hay problema, porque los numeros no se haran mas pequeños (como en la mult)
# Hay que pasarlo a logaritmo para poder hacer sumas las multiplicaciones

# Por omision a priori=0.5, es decir, proba de que sea spam es 0.5
# La c representa el caso en el que haya una palabra que no se encuentre en nuestra base de entrenamiento
# no la podemos hacer cero porque anularia todo, pero le damos un valor pequeño. 
a_posteriori <- function(trayectoria, df_entrenamiento, a_priori = 0.5, c = 1e-6)
{
  mensaje <- lee_mensaje(trayectoria)
  mensaje <- enc2utf8(mensaje)
  mensaje_TDM <- obtiene_TermDocumentMatrix(mensaje)
  conteos_mensaje <- rowSums(as.matrix(mensaje_TDM))

  # Encuentra palabras en data frame de entrenamiento
    #Palabras en comun del correo y la base
  mensaje_palabras_comunes <- intersect(names(conteos_mensaje), df_entrenamiento$terminos)
  
  # Ahora sólo aplicamos la clasificación Bayes ingenuo
  if(length(mensaje_palabras_comunes) < 1) #ninguna palabra en comun
  {
    #return(a_priori * c ^ (length(conteos_mensaje)))
    return(log(a_priori) + (length(conteos_mensaje)) *log(c)) #Aqui aplicamos el logaritmo
  }
  else
  {  #match da los indices de las palabras que si empatan
    probabilidades_palabras_comunes <- df_entrenamiento$densidad[match(mensaje_palabras_comunes, df_entrenamiento$terminos)]
    #return(a_priori * prod(probabilidades_palabras_comunes) * c ^ (length(conteos_mensaje) - length(mensaje_palabras_comunes)))
    return(log(a_priori) + sum(log(probabilidades_palabras_comunes)) + log(c) * (length(conteos_mensaje) - length(mensaje_palabras_comunes)))
  }
}

#Nota que como por la formula de BI te interesa el maximo, no hay problema en regresar el logaritmo

## Clasificación

In [50]:
clasifica_spam <- function(trayectoria, archivos) {

  hard_ham_spam_prueba <- sapply(archivos,
                             function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_spam))
  hard_ham_ham_prueba <- sapply(archivos,
                            function(p) a_posteriori(file.path(trayectoria, p), df_entrenamiento = df_easy_ham))
  
  return (ifelse(hard_ham_spam_prueba > hard_ham_ham_prueba,
                        TRUE,
                        FALSE))
}

### Hard ham: dividimos la base de datos en training y testing

In [51]:
# Leemos el directorio donde se encuentran los correos clasificados como ham dificlmente identificables
archivos_correos_hard_ham <- dir(trayectoria_hardham)

# quitamos el guión llamado cmds
archivos_correos_hard_ham <- archivos_correos_hard_ham[which(archivos_correos_hard_ham!="cmds")]

In [52]:
hard_ham_res <- clasifica_spam(trayectoria_hardham, archivos_correos_hard_ham)
easy_ham_res <- clasifica_spam(trayectoria_easyham, archivos_correos_easy_ham_testing) #Clasificamos con la base testing
spam_res     <- clasifica_spam(trayectoria_spam,    archivos_correos_spam_testing) # Clasificamos con la base testing

## Resultados

In [53]:
summary(easy_ham_res)
summary(spam_res)
summary(hard_ham_res)

   Mode   FALSE    TRUE 
logical     398       2 

   Mode   FALSE    TRUE 
logical      21     376 

   Mode   FALSE    TRUE 
logical     115     135 

### Comparación de resultados

Tipo de mail  | Resultados SIN dividir la base de datos                  |Resultados dividiendo la base de datos
------------- | ---------------------------------------------------------|-----------------------------------------------------
Easy Ham      | FALSE: 1399 / TRUE: 1                                    | FALSE: 398 / TRUE: 2
Spam          | FALSE: 21 / TRUE: 1376                                   | FALSE: 21 / TRUE: 376
Hard Ham      | FALSE: 116 / TRUE: 134                                   | FALSE: 115 / TRUE: 135

El cambio que se puede notar, tanto en Easy Ham como en Spam, es muy notorio y redundante: las respectivas cantidades de su correcta clasificación difieren en mil correos, mismos que fueron empleados en la base de entrenamiento. Dada esta obervación, se puede afirmar plenamente que el modelo sí se puede generalizar y aún así se siguen obteniendo muy buenos resultados para esta base de datos. Claro, este experimento quedá un poco limitado ya que únicamente se ha probado con un conjunto de mails. Una forma de seguir explorando la eficacia del método, sería alimentando al programa con nuevos mails.  